# Astronomy Libraries Comparison

Load AA, WWT, and Skyfield JSON outputs with pandas, parse `time` as UTC datetime, attach `astropy.time.Time`, and compare values column-by-column.


In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
from astropy.time import Time
from astropy.table import Table

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 220)



ROOT = Path('.').resolve()
OUTPUTS = ROOT / 'outputs'
FILES = {
    'aa': OUTPUTS / 'methods_to_check_aa_records.json',
    'wwt': OUTPUTS / 'methods_to_check_wwt_records.json',
    'skyfield': OUTPUTS / 'methods_to_check_skyfield_records.json',
}
ID_COLS = ['index', 'lat', 'lon', 'height', 'time', 'name']



In [ ]:
def load(key):
    df = pd.read_json(FILES[key])
    df['time'] = df['time'].apply(lambda x: Time(x))
    # df.drop(columns=['name'], inplace=True)
    return df
df_aa = load('aa')
df_wwt = load('wwt')
df_sf = load('skyfield')

In [ ]:
# --- Precision thresholds (from tests.js) ---
ANGULAR_PRECISION = 60 / 3600  # degrees (60 arcsec)
TIME_PRECISION = 60            # seconds
KM_PRECISION = 1               # km
AU_PRECISION = 100 / 1.15e8    # AU (100 km)

# --- Helper: classify columns by type ---
def classify_column(col):
    col = col.lower()

    # 360-wrap angles: RA and longitudes
    if any(x in col for x in ('rightascension', 'longitude')):
        return 'angle_circular'

    # Linear angles (no rollover)
    if any(x in col for x in ('declination', 'latitude', 'elongation', 'phaseangle')):
        return 'angle_linear'

    if 'distance' in col or 'radius' in col:
        if 'km' in col:
            return 'km'
        return 'au'

    if col.endswith('_rise') or col.endswith('_set') or col.endswith('_transit'):
        return 'time'

    return 'other'

# --- Helper: get threshold for column ---
def get_threshold(col):
    t = classify_column(col)
    if t in ('angle_circular', 'angle_linear'):
        return ANGULAR_PRECISION
    if t == 'time':
        return TIME_PRECISION / 3600
    if t == 'km':
        return KM_PRECISION
    if t == 'au':
        return AU_PRECISION
    return None

# --- Helper: compute diff with optional 360 rollover ---
def compute_diff(s1, s2, col):
    v1 = pd.to_numeric(s1, errors='coerce')
    v2 = pd.to_numeric(s2, errors='coerce')
    raw = (v1 - v2).abs()

    if classify_column(col) == 'angle_circular':
        return np.minimum(raw, 360 - raw)

    return raw

# --- Compute differences ---
def diff_and_emojis(df1, df2, label1, label2):
    result = df1[ID_COLS].copy() if all(col in df1.columns for col in ID_COLS) else pd.DataFrame(index=df1.index)
    new_cols = {}

    for col in df1.columns:
        if col in ID_COLS:
            continue
        if 'pluto' in col:
            continue

        threshold = get_threshold(col)
        diff = compute_diff(df1[col], df2[col], col)

        new_cols[col] = [
            {label1: v1, label2: v2, 'diff': d}
            for v1, v2, d in zip(df1[col], df2[col], diff)
        ]

        if threshold is not None:
            both_nan = pd.isna(df1[col]) & pd.isna(df2[col])
            pass_col = (diff <= threshold) | both_nan
            emoji_col = pass_col.map(lambda x: '✅' if x else '❌')
            new_cols[col + '_emoji'] = emoji_col

    result = pd.concat([result, pd.DataFrame(new_cols, index=df1.index)], axis=1)
    id_cols = [col for col in ID_COLS if col in result.columns]
    other_cols = [col for col in result.columns if col not in id_cols]
    result = result[id_cols + other_cols]
    return result

def only_emoji(df):
    cols = [c for c in df.columns if 'emoji' in c]
    return df[cols]

def filter_red_x(df):
    emoji_cols = [c for c in df.columns if c.endswith('_emoji')]
    # Any row with at least one ❌ in any emoji col
    mask = df[emoji_cols].apply(lambda row: '❌' in row.values, axis=1)
    filtered = df[mask].copy()
    # Drop non-ID columns (and their _emoji) that have no ❌ in any row
    cols_to_keep = list(ID_COLS)
    for col in df.columns:
        if col in ID_COLS or not col.endswith('_emoji'):
            continue
        if (filtered[col] == '❌').any():
            base_col = col[:-6]  # remove _emoji
            cols_to_keep.append(base_col)
            cols_to_keep.append(col)
    # Keep only ID columns and those with at least one ❌
    cols_to_keep = [c for c in cols_to_keep if c in filtered.columns]
    return filtered[cols_to_keep]


In [ ]:
sf_aa = filter_red_x(diff_and_emojis(df_sf, df_aa, 'sf', 'aa'))
# Table.from_pandas(sf_aa).show_in_browser(jsviewer=True)
print(f"{sf_aa.to_numpy().flatten().tolist().count('❌')} errors")

In [ ]:
sf_wwt = filter_red_x(diff_and_emojis(df_sf, df_wwt, 'sf', 'wwt'))
# Table.from_pandas(sf_wwt).show_in_browser(jsviewer=True)
print(f"{sf_wwt.to_numpy().flatten().tolist().count('❌')} errors")


In [ ]:
aa_wwt = filter_red_x(diff_and_emojis(df_aa, df_wwt, 'aa', 'wwt'))
# Table.from_pandas(sf_wwt).show_in_browser(jsviewer=True)
print(f"{aa_wwt.to_numpy().flatten().tolist().count('❌')} errors")

In [ ]:
Table.from_pandas(sf_wwt).write('sf_wwt.html', format='ascii.html', overwrite=True)
Table.from_pandas(sf_aa).write('sf_aa.html', format='ascii.html', overwrite=True)
Table.from_pandas(aa_wwt).write('aa_wwt.html', format='ascii.html', overwrite=True)